In [2]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import random

In [3]:
def seconds(time_str):
    [hrs,mins,secs] = list(map(int,time_str.split(":")))
    # hrs = hrs%24
    return np.mod(int(hrs*120 + mins*2 + secs/30),2880)

In [4]:
df = pd.read_csv('google_transit/stop_times.txt')
df['day'] = df.apply(lambda row: row['trip_id'].split("-")[2], axis=1)
df['arrival_time'] = df.apply(lambda row: seconds(row['arrival_time']), axis=1)
df['departure_time'] = df.apply(lambda row: seconds(row['departure_time']), axis=1)

In [5]:
stops = pd.read_csv('google_transit/stops.txt')

In [6]:
stops['coords'] = stops.apply(lambda row: (row['stop_lat'],row['stop_lon']), axis=1)
stops['stop_id'] = stops.apply(lambda row: row['stop_id'][0:3], axis=1)
coords = dict(zip(stops['stop_id'],stops['coords']))

In [7]:
stations = df['stop_id'].unique()
stations = np.append(stations,'140N') # add south ferry
stations = np.append(stations,'140S') # add south ferry

times = [i for i in range(120*24)]

In [8]:
trans = pd.read_csv('google_transit/transfers.txt')
trans['from_stop_id'] = trans['from_stop_id'].apply(str)
trans['to_stop_id'] = trans['to_stop_id'].apply(str)

# make a transfer dictionary
trans_dict = dict(zip(trans['from_stop_id'],zip(trans['to_stop_id'],trans['min_transfer_time'])))

In [9]:
# go through the dataframe, if two stops share a trip_id connect the earlier time to the later time
time_graph = {}

past_stop_id = df['stop_id'][0]
past_time = df['departure_time'][0]
past_trip_id = df['trip_id'][0]
counter = 0

for i in range(1,len(df)):
    current_stop_id = df['stop_id'][i]
    current_time = df['arrival_time'][i]
    current_trip_id = df['trip_id'][i]

    # connect stations you can ride between
    if current_trip_id==past_trip_id:
        if (past_stop_id, past_time) in time_graph:
            time_graph[(past_stop_id, past_time)].append(((current_stop_id, current_time),'train'))
        else:
            time_graph[(past_stop_id, past_time)] = [((current_stop_id, current_time),'train')]

    # to deal with transfers, note that there is never a scenario where you would wait at a station and only then transfer
    # (it would always be done immediately after arrival)
    # so transfers only have to connect directly after trains arrive

    # if a transfer exists
    if current_stop_id[0:3] in trans_dict:
        transfer_stop_id = trans_dict[current_stop_id[0:3]][0]

        # if youre transferring between north and south stations
        if current_stop_id[0:3]==transfer_stop_id:
            dir = current_stop_id[3]

            if dir=='N':
                transfer_stop_id += 'S'
            
            if dir=='S':
                transfer_stop_id += 'N'

            transfer_time = int(trans_dict[current_stop_id[0:3]][1]/30)

            if (current_stop_id, current_time) in time_graph:
                time_graph[(current_stop_id, current_time)].append(((transfer_stop_id, np.mod(current_time+transfer_time,2880)),'transfer'))
            else:
                time_graph[(current_stop_id, current_time)] = [((transfer_stop_id, np.mod(current_time+transfer_time,2880)),'transfer')]
        else:
        # if youre transferring between other stations, go to either north or south? data is not more specific
            transfer_time = int(trans_dict[current_stop_id[0:3]][1]/30)

            if (current_stop_id, current_time) in time_graph:
                time_graph[(current_stop_id, current_time)].append(((transfer_stop_id+'S', np.mod(current_time+transfer_time,2880)),'transfer'))
                time_graph[(current_stop_id, current_time)].append(((transfer_stop_id+'N', np.mod(current_time+transfer_time,2880)),'transfer'))
            else:
                time_graph[(current_stop_id, current_time)] = [((transfer_stop_id+'S', np.mod(current_time+transfer_time,2880)),'transfer')]
                time_graph[(current_stop_id, current_time)] = [((transfer_stop_id+'N', np.mod(current_time+transfer_time,2880)),'transfer')]

    past_stop_id = current_stop_id
    past_time = df['departure_time'][i]
    past_trip_id = current_trip_id

In [10]:
time_graph

{('101S', 13): [(('103S', 16), 'train')],
 ('103S', 16): [(('103N', 22), 'transfer'), (('104S', 19), 'train')],
 ('104S', 19): [(('104N', 25), 'transfer'), (('106S', 22), 'train')],
 ('106S', 22): [(('106N', 28), 'transfer'), (('107S', 25), 'train')],
 ('107S', 25): [(('107N', 31), 'transfer'), (('108S', 27), 'train')],
 ('108S', 27): [(('108N', 33), 'transfer'), (('109S', 30), 'train')],
 ('109S', 30): [(('109N', 36), 'transfer'), (('110S', 33), 'train')],
 ('110S', 33): [(('110N', 39), 'transfer'), (('111S', 36), 'train')],
 ('111S', 36): [(('111N', 42), 'transfer'), (('112S', 40), 'train')],
 ('112S', 40): [(('A09N', 46), 'transfer'), (('113S', 43), 'train')],
 ('113S', 43): [(('113N', 49), 'transfer'), (('114S', 47), 'train')],
 ('114S', 47): [(('114N', 53), 'transfer'), (('115S', 50), 'train')],
 ('115S', 50): [(('115N', 56), 'transfer'), (('116S', 53), 'train')],
 ('116S', 53): [(('116N', 59), 'transfer'), (('117S', 57), 'train')],
 ('117S', 57): [(('117N', 63), 'transfer'), (('1

In [11]:
# ignoring running, you would only ever wait at a station until another event happens at that same station
# so we should find every edge entering or leaving each station, order them in time, and connect them all in time

# TEMPORARY FIX: connect every node to itself in time :(

for station in stations:
    for time in times:
        if (station, time) in time_graph:
            time_graph[(station, time)].append(((station, np.mod(time+1,2880)),'wait'))
        else:
            time_graph[(station, time)] = [((station, np.mod(time+1,2880)),'wait')]

Approximate solution: Branch and Bound (B&B)

Idea is to perform a DFS to a predefined depth which is our lookahead parameter. For all of the generated paths, we compute a heuristic like alpha*(path complexity) - beta*(makes you run a bunch) + gamma*(number of stations visited) and pick the path with the largest heuristic value. Then we jump to the end of this route and complete the same process again

In [243]:
class PathFinder():
    def __init__(self) -> None:
        self.paths = []
        self.visited = set()

    def DFS(self, node, depth, path, lookahead):
        if depth>lookahead:
            self.paths.append(path.copy())
            return

        path.append(node)

        for neighbor in time_graph[node]:
            self.DFS(neighbor,depth+1,path,lookahead)

            if len(path)>1:
                path.pop()

        return

    def do_DFS(self, starting_node, lookahead=50):
        self.DFS(node=starting_node, depth=0, path=[], lookahead=lookahead)

    def rate(self):
        # how many unique stations do the paths encounter?
        station_count = []
        for path in self.paths:
            seen = set()
            for station_id in path:
                station = station_id[0][0:3]
                # print(station)
                if station not in seen:
                    seen.add(station)
            station_count.append(len(seen))
        
        best_station_count = np.argsort(station_count)

        # waiting penalty: how long does the path spend waiting at a single station
        # waiting_penalty = []
        # for path in self.paths:
        #     past_station = path[0]

        return best_station_count[-100:]

finder = PathFinder()
finder.do_DFS(starting_node=('138N', 500), lookahead=60)
best = finder.rate()

Above doesn't really work. A different approach to generate viable paths: perform a Monte Carlo simulation of different possible paths to take (bypasses the DFS and BFS problem of taking too many express lines and transfers)

At each step there are predefined probabilities to perform certain actions: i.e. stay on the same line, transfer, wait, take the express line, etc.

In [86]:
class MonteCarlo():
    def __init__(self) -> None:
        self.paths = []
        self.visited_stations = set()
        self.time_visited_stations = {}

    def choose_next(self, current_node):
        p1 = 0.95 # probability that you take a train if you see one
        p2 = 0.5 # probability that you take it to a train to a station youve already been to
                 # if there are no other trains (waiting=bad)
        p3 = 0.2 # probability of transferring over waiting

        #print(current_node)
        neighbors = time_graph[current_node]
        random.shuffle(neighbors)
        types = [neighbor[1] for neighbor in neighbors]

        train_i = [i for i,type in enumerate(types) if type=='train' ]
        train_options = [neighbors[i][0] for i in train_i]
        train_options_w_time = [neighbors[i] for i in train_i]
                
        if 'train' in types:
            not_visited = [train_option for train_option in train_options if train_option[0][0:3] not in self.visited_stations]
            visited = [train_option for train_option in train_options if train_option[0][0:3] in self.visited_stations]

            # are any of the train options places we haven't yet been to?
            if len(not_visited)>0:
                #get one of them
                train_option = not_visited[0]

                if random.random()<p1:
                    # print("TRAIN!")
                    # adds station to set of those we've visited
                    self.visited_stations.add(train_option[0][0:3])

                    # records time at which we visited (will help for optimization)
                    self.time_visited_stations[train_option[0][0:3]] = train_options_w_time[0][0][1]
                    
                    return train_option
                else:
                    pass
            else:
                train_option = visited[0]
                if random.random()<p2:
                    # print("sad TRAIN!")
                    return train_option
                else:
                    pass
        
        if 'transfer' in types:
            transfer_i = [i for i,type in enumerate(types) if type=='transfer' ]
            transfer_options = [neighbors[i][0] for i in transfer_i]

            wait_i = [i for i,type in enumerate(types) if type=='wait' ]
            wait_options = [neighbors[i][0] for i in wait_i]

            if random.random()<p2:
                # print("TRANSFER")
                return transfer_options[0]
            else:
                # print("WAIT")
                return wait_options[0]
        else:
            wait_i = [i for i,type in enumerate(types) if type=='wait' ]
            wait_options = [neighbors[i][0] for i in wait_i]

            # print("WAIT")
            return wait_options[0]

    def generate_path(self, starting_node, length=1):
        path = []
        self.visited_stations = set()
        self.time_visited_stations = {}
        current_node = starting_node

        for i in range(length):
            current_node = self.choose_next(current_node)
            path.append(current_node)

        #self.paths.append(path)
        return path

    def ensemble(self, starting_node=('H11N', 500), num_candidates=100, length_candidates=10000):
        scores = []

        for i in range(num_candidates):
            path = self.generate_path(starting_node, length=length_candidates)
            self.paths.append(path)
            scores.append(len(self.visited_stations))

        return scores

    def plot_route(self, route):
        lons = [coord[1] for coord in coords.values()]
        lats = [coord[0] for coord in coords.values()]

        plt.scatter(lons,lats,s=0.1,c='b')

        route_lons = [coords[station[0][0:3]][1] for station in route]
        route_lats = [coords[station[0][0:3]][0] for station in route]

        plt.scatter(route_lons,route_lats,s=0.3,c='r')

    def modify(self,path):
        # takes a random node along the path of the path and seeds a new path starting there.
        # Connect back to the main route when it gets to a station that the original path will
        # see but has not yet arrived at. There we wait...

        N = len(path)
        m = random.randint(0,N-1)
        starting_node = path[m]

        old_visited_stations = self.visited_stations.copy()
        old_time_visited_stations = self.time_visited_stations.copy()

        new_path = self.generate_path(starting_node, length=N-m)
        new_path = new_path[1:]

        for station,time in new_path:
            if station[0:3] in old_visited_stations:
                time_to_get = old_time_visited_stations[station[0:3]]
                if time<time_to_get:
                    print("FOUND")
                    print(time)
                    time_found = time  #time where we found the station

                    waiting_path = []
                    i = time
                    while i<time_to_get:
                        waiting_path.append((station,i))
                        i+=1

                    print(waiting_path)
                    
                    final_path = path[0:m] + new_path[m:time_found] + waiting_path + path[i:]
                    
                    return path[0:m] + waiting_path
                else:
                    pass
            else:
                pass

        return path

    def anneal(self, starting_node):
        # generate first path
        path = self.generate_path(starting_node, length=10000)

        #while True:
            # generate new path by selecting a random element 
            # of the current path and seeding a new path

    
finder = MonteCarlo()
#scores = finder.ensemble(num_candidates=500)

path = finder.generate_path(('H11N', 500), length=10000)
#finder.plot_route(1)

new_path = finder.modify(path)
# scores = -np.sort([-score for score in scores])

FOUND
1493
[('Q05N', 1493), ('Q05N', 1494), ('Q05N', 1495), ('Q05N', 1496), ('Q05N', 1497), ('Q05N', 1498), ('Q05N', 1499), ('Q05N', 1500), ('Q05N', 1501), ('Q05N', 1502), ('Q05N', 1503), ('Q05N', 1504), ('Q05N', 1505), ('Q05N', 1506), ('Q05N', 1507), ('Q05N', 1508), ('Q05N', 1509), ('Q05N', 1510), ('Q05N', 1511), ('Q05N', 1512), ('Q05N', 1513), ('Q05N', 1514), ('Q05N', 1515), ('Q05N', 1516), ('Q05N', 1517), ('Q05N', 1518), ('Q05N', 1519), ('Q05N', 1520), ('Q05N', 1521), ('Q05N', 1522), ('Q05N', 1523), ('Q05N', 1524), ('Q05N', 1525), ('Q05N', 1526), ('Q05N', 1527), ('Q05N', 1528), ('Q05N', 1529), ('Q05N', 1530), ('Q05N', 1531), ('Q05N', 1532), ('Q05N', 1533), ('Q05N', 1534), ('Q05N', 1535), ('Q05N', 1536), ('Q05N', 1537), ('Q05N', 1538), ('Q05N', 1539), ('Q05N', 1540), ('Q05N', 1541), ('Q05N', 1542), ('Q05N', 1543), ('Q05N', 1544), ('Q05N', 1545), ('Q05N', 1546), ('Q05N', 1547), ('Q05N', 1548), ('Q05N', 1549), ('Q05N', 1550), ('Q05N', 1551), ('Q05N', 1552), ('Q05N', 1553), ('Q05N', 155

In [87]:
new_path

[('Q05N', 1493),
 ('Q05N', 1494),
 ('Q05N', 1495),
 ('Q05N', 1496),
 ('Q05N', 1497),
 ('Q05N', 1498),
 ('Q05N', 1499),
 ('Q05N', 1500),
 ('Q05N', 1501),
 ('Q05N', 1502),
 ('Q05N', 1503),
 ('Q05N', 1504),
 ('Q05N', 1505),
 ('Q05N', 1506),
 ('Q05N', 1507),
 ('Q05N', 1508),
 ('Q05N', 1509),
 ('Q05N', 1510),
 ('Q05N', 1511),
 ('Q05N', 1512),
 ('Q05N', 1513),
 ('Q05N', 1514),
 ('Q05N', 1515),
 ('Q05N', 1516),
 ('Q05N', 1517),
 ('Q05N', 1518),
 ('Q05N', 1519),
 ('Q05N', 1520),
 ('Q05N', 1521),
 ('Q05N', 1522),
 ('Q05N', 1523),
 ('Q05N', 1524),
 ('Q05N', 1525),
 ('Q05N', 1526),
 ('Q05N', 1527),
 ('Q05N', 1528),
 ('Q05N', 1529),
 ('Q05N', 1530),
 ('Q05N', 1531),
 ('Q05N', 1532),
 ('Q05N', 1533),
 ('Q05N', 1534),
 ('Q05N', 1535),
 ('Q05N', 1536),
 ('Q05N', 1537),
 ('Q05N', 1538),
 ('Q05N', 1539),
 ('Q05N', 1540),
 ('Q05N', 1541),
 ('Q05N', 1542),
 ('Q05N', 1543),
 ('Q05N', 1544),
 ('Q05N', 1545),
 ('Q05N', 1546),
 ('Q05N', 1547),
 ('Q05N', 1548),
 ('Q05N', 1549),
 ('Q05N', 1550),
 ('Q05N', 1551

In [59]:
path==new_path

False

In [16]:
sorted_scores = -np.sort([-score for score in scores])
ids = np.argsort(scores)
sorted_scores

array([193, 172, 171, 168, 165, 165, 161, 161, 160, 160, 156, 156, 155,
       155, 154, 154, 154, 153, 153, 153, 152, 149, 149, 149, 148, 147,
       146, 146, 146, 145, 145, 145, 144, 144, 144, 144, 142, 142, 142,
       142, 141, 141, 140, 140, 140, 137, 137, 137, 137, 137, 136, 136,
       136, 136, 136, 136, 135, 135, 135, 135, 134, 134, 134, 134, 133,
       133, 133, 133, 133, 132, 132, 132, 132, 132, 132, 131, 130, 130,
       130, 130, 130, 129, 129, 128, 128, 127, 127, 127, 127, 127, 126,
       126, 125, 125, 125, 125, 125, 125, 124, 124, 123, 123, 122, 121,
       121, 120, 120, 120, 120, 119, 119, 118, 118, 118, 118, 117, 117,
       117, 117, 117, 117, 117, 116, 116, 116, 116, 116, 115, 115, 115,
       115, 114, 114, 114, 114, 114, 113, 113, 113, 112, 112, 112, 112,
       112, 112, 111, 111, 111, 111, 110, 110, 110, 110, 110, 109, 109,
       109, 109, 108, 108, 108, 108, 108, 108, 108, 107, 107, 107, 107,
       107, 107, 107, 107, 106, 106, 106, 106, 105, 105, 105, 10

In [63]:
finder.plot_route(new_path)
finder.plot_route(path)

TypeError: plot_route() takes 2 positional arguments but 3 were given

In [458]:
finder.paths[ids[-1]]

[('H11N', 501),
 ('H11N', 502),
 ('H11N', 503),
 ('H11N', 504),
 ('H11N', 505),
 ('H11N', 506),
 ('H11N', 507),
 ('H11N', 508),
 ('H11N', 509),
 ('H11N', 510),
 ('H11N', 511),
 ('H11N', 512),
 ('H11N', 513),
 ('H11N', 514),
 ('H11N', 515),
 ('H11N', 516),
 ('H11N', 517),
 ('H11N', 518),
 ('H11N', 519),
 ('H11N', 520),
 ('H11N', 521),
 ('H11N', 522),
 ('H11N', 523),
 ('H11N', 524),
 ('H10N', 527),
 ('H09N', 531),
 ('H08N', 534),
 ('H07N', 538),
 ('H06N', 541),
 ('H04N', 549),
 ('H03N', 562),
 ('H02N', 566),
 ('H01N', 568),
 ('H01N', 569),
 ('H01N', 570),
 ('H01N', 571),
 ('H01N', 572),
 ('H01N', 573),
 ('H01N', 574),
 ('H01N', 575),
 ('H01N', 576),
 ('H01N', 577),
 ('H01N', 578),
 ('H01N', 579),
 ('H01N', 580),
 ('H01N', 581),
 ('H01N', 582),
 ('H01N', 583),
 ('H01N', 584),
 ('H01N', 585),
 ('H01N', 586),
 ('H01N', 587),
 ('H01N', 588),
 ('H01N', 589),
 ('H01N', 590),
 ('H01N', 591),
 ('H01N', 592),
 ('H01N', 593),
 ('H01N', 594),
 ('H01N', 595),
 ('H01N', 596),
 ('H01N', 597),
 ('H01N'

In [375]:
scores

array([222, 215, 202, 200, 199, 198, 197, 196, 195, 195, 195, 194, 194,
       193, 192, 191, 190, 190, 188, 187, 184, 182, 182, 181, 180, 179,
       179, 179, 179, 179, 178, 178, 177, 177, 177, 177, 177, 176, 176,
       175, 174, 174, 172, 172, 172, 172, 171, 171, 171, 170, 170, 170,
       170, 170, 170, 169, 169, 169, 169, 169, 168, 168, 168, 168, 167,
       167, 167, 166, 166, 166, 164, 164, 164, 163, 163, 163, 162, 162,
       162, 162, 162, 162, 161, 161, 161, 161, 161, 161, 160, 160, 160,
       160, 160, 160, 159, 159, 158, 158, 158, 158, 158, 157, 157, 157,
       157, 157, 157, 157, 156, 156, 156, 156, 156, 156, 155, 155, 155,
       155, 155, 155, 155, 155, 154, 154, 154, 154, 154, 154, 154, 154,
       154, 154, 153, 153, 153, 153, 153, 153, 152, 152, 151, 151, 151,
       151, 151, 151, 150, 150, 150, 150, 150, 150, 150, 150, 150, 149,
       149, 149, 149, 149, 148, 148, 148, 148, 148, 147, 147, 147, 147,
       147, 147, 147, 146, 146, 146, 146, 146, 146, 146, 146, 14

In [343]:
print(finder.visited_stations)

{'104', 'A47', 'F12', 'A55', 'D18', 'L21', '133', 'A40', 'H08', '113', '112', 'A63', 'L15', '114', 'G28', 'L22', 'L16', 'M10', 'M18', 'H09', 'G33', 'M14', 'A31', 'A34', 'D19', 'A61', 'A24', 'L26', 'G35', 'H11', '128', '122', 'A42', 'A44', 'A51', 'L20', '119', 'M16', 'A28', 'F09', '120', '135', '108', 'H04', 'H10', '107', 'L14', 'A46', '126', 'A32', 'L17', '131', 'A15', '129', 'L13', 'G24', 'L19', 'D21', '123', 'A33', '110', 'A12', 'A30', '115', '101', '130', 'A38', 'D20', 'M09', 'A45', 'A48', 'L27', 'G26', 'M13', 'M12', '136', 'A57', 'L28', 'L12', 'H02', 'L24', '103', '106', 'H07', 'L11', 'D15', 'G34', 'A41', 'A64', '127', 'L25', '111', '134', '117', 'H06', 'G32', 'G22', '132', '125', 'H03', 'G29', 'M11', 'A36', 'D16', '124', 'A59', 'G31', '137', '109', 'F11', '121', 'A65', 'L29', '116', 'H01', 'A27', 'A60', 'L10', 'A43', 'D17', 'G30', '118', 'G36'}


In [322]:
time_graph[('121', 500)]

KeyError: ('121', 500)

In [303]:
l = time_graph[('138N', 500)].copy()

In [304]:
l

[(('138N', 501), 'wait')]

In [244]:
len(finder.paths)

271873

In [245]:
best

array([226386,  25374,  55932,  97606,  46294, 205444,  46293,  97605,
        98948,  59685, 255975,  98951,  97604, 179575,  46170,  21636,
       174494,  64029, 118381,  74217, 252841,  74220, 102082, 185585,
        42944,  17006, 102079,  97609,  17009,  54315,  97614,  83867,
        52652, 177219, 233107, 177223,  97611,  97610, 113415,  60635,
       228838, 179571, 129665, 135291, 179320,  37030,   5952, 140540,
        22776,  48380,  89271,  22779, 177834, 174042, 177838, 192087,
       213725, 146644, 146646, 253257,  31200,  47734, 268003,  47736,
        68447,  68450,   9657, 189500, 161147, 177583,  77615, 253044,
       192539,   8249, 110999,  53079, 115566,   5858,  92837,   1987,
       226515, 172405, 149221, 224364, 132749,  97601, 244319,  74216,
        46162,  13976, 161146, 164796, 160910, 160909, 103221, 160908,
        22775, 218491, 244320,  97603])

In [246]:
finder.paths[97603]
# seen = set()
# for station_id in path:
#     station = station_id[0][0:3]
#     # print(station)
#     if station not in seen:
#         seen.add(station)
# print(len(seen))

[('138N', 500),
 ('226N', 593),
 ('225N', 596),
 ('224N', 599),
 ('221N', 609),
 ('221S', 615),
 ('221S', 616),
 ('221S', 617),
 ('221S', 618),
 ('222S', 623),
 ('415N', 629),
 ('415N', 630),
 ('415N', 631),
 ('415N', 632),
 ('415N', 633),
 ('415N', 634),
 ('415N', 635),
 ('415N', 636)]

In [141]:
max_val = 0
for id,val in enumerate(visited):
    if val[1]>max_val:
        max_val = val[1]
        print(val)

('101S', 13)
('103S', 16)
('103N', 22)
('103N', 23)
('104N', 25)
('103S', 29)
('103S', 30)
('107N', 31)
('107N', 32)
('108N', 33)
('108N', 34)
('109N', 36)
('109N', 37)
('110N', 39)
('107S', 40)
('111N', 42)
('106S', 43)
('A09N', 46)
('111S', 49)
('111S', 50)
('110S', 52)
('114N', 53)
('109S', 54)
('115N', 56)
('108S', 57)
('116N', 59)
('107S', 60)
('117N', 63)
('117N', 64)
('118N', 65)
('104S', 66)
('119N', 67)
('103S', 69)
('A03S', 70)
('120N', 71)
('A05N', 72)
('121N', 75)
('A06N', 76)
('122N', 77)
('A07N', 78)
('107S', 80)
('124N', 83)
('A07S', 85)
('124S', 89)
('124S', 90)
('A05S', 91)
('127S', 93)
('122S', 94)
('A03S', 96)
('R16N', 99)
('128N', 100)
('128N', 101)
('119S', 103)
('A17N', 104)
('118S', 105)
('117S', 107)
('A15N', 108)
('A07N', 109)
('116S', 112)
('D13N', 115)
('R14S', 116)
('D12N', 118)
('R11N', 125)
('R11N', 126)
('D11S', 127)
('D11S', 128)
('D10S', 132)
('D10S', 133)
('D09S', 136)
('D09S', 137)
('D08S', 140)
('D08S', 141)
('D08S', 142)
('R30S', 150)
('R30N', 156)
